In [1]:
import pandas as pd
from nsepy import get_history as gh
import datetime as dt
import yfinance as yf
import datetime as dt

In [2]:
start = dt.datetime(2021, 7, 1)
end = dt.datetime(2022, 1, 27)
symbol = 'AXISBANK.NS'  # Stock symbol in uppercase

stk_data = yf.download(symbol, start=start, end=end)

[*********************100%%**********************]  1 of 1 completed


In [3]:
stk_data=stk_data[["Open","High","Low","Close"]]
dataset=stk_data

In [4]:
from sklearn.preprocessing import MinMaxScaler
Ms = MinMaxScaler()
data1= Ms.fit_transform(stk_data)
print("Len:",data1.shape)
#column="Close"

Len: (143, 4)


In [5]:
data1=pd.DataFrame(data1,columns=["Open","High","Low","Close"])
data1

,Open,High,Low,Close
0,0.508982,0.441549,0.543815,0.491495
1,0.497006,0.433854,0.540997,0.513145
2,0.544411,0.479275,0.593970,0.563144
3,0.543912,0.504095,0.604114,0.552835
4,0.538423,0.489203,0.587489,0.576289
...,...,...,...,...
138,0.385230,0.330107,0.398422,0.404639
139,0.414421,0.355920,0.417864,0.401031
140,0.369261,0.296848,0.338405,0.316495
141,0.334331,0.252916,0.226543,0.275773


In [6]:
training_size = round(len(data1 ) * 0.80)
print(training_size)
X_train=data1[:training_size]
X_test=data1[training_size:]
print("X_train length:",X_train.shape)
print("X_test length:",X_test.shape)
y_train=data1[:training_size]
y_test=data1[training_size:]
print("y_train length:",y_train.shape)
print("y_test length:",y_test.shape)

114
X_train length: (114, 4)
X_test length: (29, 4)
y_train length: (114, 4)
y_test length: (29, 4)


In [7]:
import warnings
warnings.filterwarnings("ignore")

In [8]:
listt=["Close","High","Open","Low"]
#listt=["AQI_calculated","PM10","PM2.5","NOx","NO2","NO","NH3","SO2","CO",'year']

In [9]:
performance={"Model":[],"RMSE":[],"MaPe":[],"Lag":[],"Test":[]}

In [10]:
def combination(data1, listt):
    # Print the list of variables being used
    print("Variables used:", listt)
    
    # Extract the subset of the dataset with the selected variables
    datasetTwo = dataset[listt]
    
    # Define the number of observations for testing
    test_obs = 28
    
    # Split the data into training and testing sets
    train = datasetTwo[:-test_obs]
    test = datasetTwo[-test_obs:]
    
    # Import necessary libraries
    from statsmodels.tsa.statespace.varmax import VARMAX  
    from itertools import product
    
    # Initialize variables for storing best parameters
    best_aic = float('inf')
    best_params = None

    # Iterate over combinations of p and q values
    for p, q in product(range(1, 4), range(1, 4)):
        try:
            # Fit VARMAX model with current order
            model = VARMAX(train, order=(p, q))
            results = model.fit(maxiter=1000, disp=False)
            
            # Update best parameters if current AIC is lower
            aic = results.aic
            if aic < best_aic:
                best_aic = aic
                best_params = (p, q)
        except:
            continue

    # Print the best AIC and corresponding parameters
    print("Best AIC:", best_aic)
    print("Best Parameters (p, q):", best_params)

    # Fit VARMAX model with the best parameters
    model = VARMAX(train, order=best_params) 
    results = model.fit()
    
    # Forecast
    lagged_values = train.values[-max(best_params):]
    pred = results.forecast(y=lagged_values, steps=test_obs) 
    preds = pd.DataFrame(pred, columns=listt)
    preds.to_csv("varforecasted_{}.csv".format(test_obs))
    
    # Calculate performance metrics
    from sklearn.metrics import mean_squared_error, mean_absolute_percentage_error
    rmse = round(mean_squared_error(test, pred, squared=False))
    mape = mean_absolute_percentage_error(test, pred)
    
    # Store performance metrics in a dictionary
    performance = {
        "Model": [listt],
        "RMSE": [rmse],
        "MaPe": [mape],
        "Lag": [best_params],
        "Test": [test_obs]
    }
    
    # Create a DataFrame with performance metrics and return
    perf = pd.DataFrame(performance)
    return perf, results, pred

In [11]:
perf,result,pred= combination(data1, listt)

Variables used: ['Close', 'High', 'Open', 'Low']


C:\Users\sreet\Documents\anaconda installation\envs\aidl\lib\site-packages\statsmodels\tsa\statespace\varmax.py:160: EstimationWarning: Estimation of VARMA(p,q) models is not generically robust, due especially to identification issues.
  warn('Estimation of VARMA(p,q) models is not generically robust,'
C:\Users\sreet\Documents\anaconda installation\envs\aidl\lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
C:\Users\sreet\Documents\anaconda installation\envs\aidl\lib\site-packages\statsmodels\tsa\statespace\varmax.py:160: EstimationWarning: Estimation of VARMA(p,q) models is not generically robust, due especially to identification issues.
  warn('Estimation of VARMA(p,q) models is not generically robust,'
C:\Users\sreet\Documents\anaconda installation\envs\aidl\lib\site-packages\statsmodels\tsa\base\tsa_model.p

Best AIC: 2943.481023877189
Best Parameters (p, q): (2, 1)


C:\Users\sreet\Documents\anaconda installation\envs\aidl\lib\site-packages\statsmodels\tsa\statespace\varmax.py:160: EstimationWarning: Estimation of VARMA(p,q) models is not generically robust, due especially to identification issues.
  warn('Estimation of VARMA(p,q) models is not generically robust,'
C:\Users\sreet\Documents\anaconda installation\envs\aidl\lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
C:\Users\sreet\Documents\anaconda installation\envs\aidl\lib\site-packages\statsmodels\base\model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "
C:\Users\sreet\Documents\anaconda installation\envs\aidl\lib\site-packages\statsmodels\tsa\base\tsa_model.py:836: ValueWarning: No supported index is avai

In [12]:
pred

,Close,High,Open,Low
115,709.415524,717.979765,709.974713,702.031347
116,711.345292,718.321058,710.903703,702.769606
117,712.446546,720.191777,712.444507,705.533767
118,713.794368,721.572185,713.495831,706.750583
119,714.986972,722.600388,714.824655,707.709857
120,715.971701,723.728357,716.010686,708.841458
121,716.989270,724.753004,717.000108,709.838691
122,717.982292,725.740831,718.022265,710.771219
123,718.916829,726.710357,719.015308,711.715316
124,719.814999,727.641440,719.952482,712.600641


In [13]:
perf

,Model,RMSE,MaPe,Lag,Test
0,"[Close, High, Open, Low]",29,0.034363,"(2, 1)",28
